In [4]:
#!pip install yahoo-fin
!pip install tweepy

  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.26.0
    Uninstalling requests-2.26.0:
      Successfully uninstalled requests-2.26.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.


#### imports

In [5]:
from yahoo_fin import stock_info as si
import tweepy
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [6]:
ticker_symbols = si.tickers_sp500()

## Scraper Class:

#### Constructor takes 2 arguments
1. (string) : Stock ticker symbol
2.  (int)   : Number of pages to scrape from twitter (each page returns 100 tweets, may include retweets)

#### get_tweets() method
returns tweets


In [57]:
client = tweepy.Client(bearer_token="AAAAAAAAAAAAAAAAAAAAAJufYQEAAAAAKav7aNECQA0fFGEwRJ9Xre6x5uo%3Db1ceEnW5RwZTqNlXKInTamgCMDKJabKBafw6H2tUKY0qqruZby")
lemmatizer = WordNetLemmatizer()

# Class object for scrapping
class Scraper:
    def __init__(self, stock_ticker,pages ):
        self.ticker = stock_ticker
        self.data = []
        # Filling page results
        self.__scrape_pages(None, pages, self.data)
        self.__get_text()

    def __scrape_pages(self, nextToken, pages, responses):
        if pages == 0:
            return
        query = f"#{self.ticker} lang:en"
        tweets = client.search_recent_tweets(query=query,
                                         tweet_fields = ["text", "public_metrics"],
                                         max_results = 100,
                                         expansions='author_id',
                                         next_token=nextToken
                                         )
        responses.extend(tweets.data)
        # recursive call
        self.__scrape_pages(tweets[3]['next_token'], pages - 1, responses)
        
        
    def __get_text(self):
        text = []
        for data in self.data:
            text.append(data.text)
        self.data = text
        
        
    def __clean_text(self, data):    
        extra_stop = set(())
        stoplist = set(stopwords.words('english')) | extra_stop


        cleaned_data = data.copy()
        # Remove Mentions
        cleaned_data = [(" ".join(filter(lambda x:x[0]!='@', tweet.split()))) for tweet in cleaned_data]
        # Remove HTTPS
        cleaned_data = [(re.sub(r'http\S+', '', tweet)) for tweet in cleaned_data]
        # Remove Special Chars
        cleaned_data = [re.sub("[^a-zA-Z]+", " ", tweet) for tweet in cleaned_data]
        # Remove Stop Words

        cleaned_data = [[lemmatizer.lemmatize(word) for word in tweet.lower().split() if word not in stoplist and len(word) > 1]
                           for tweet in cleaned_data]

        cleaned_data = [" ".join(word) for word in cleaned_data]

        return cleaned_data        

    
    def get(self, no_retweets=True, clean_data=True):
        if no_retweets:
            temp_data = []
            for data in self.data:
                if "RT @" not in data:
                    temp_data.append(data)
        else:
            temp_data = self.data
            
        
        if clean_data:
            temp_data = self.__clean_text(temp_data)
        
        
        return temp_data
        

In [58]:
def clean_text(data):
    
    extra_stop = set(())
    stoplist = set(stopwords.words('english')) | extra_stop
    
    
    cleaned_data = data.copy()
    # Remove Mentions
    cleaned_data = [(" ".join(filter(lambda x:x[0]!='@', tweet.split()))) for tweet in cleaned_data]
    # Remove HTTPS
    cleaned_data = [(re.sub(r'http\S+', '', tweet)) for tweet in cleaned_data]
    # Remove Special Chars
    cleaned_data = [re.sub("[^a-zA-Z]+", " ", tweet) for tweet in cleaned_data]
    # Remove Stop Words
    cleaned_data = [[lemmatizer.lemmatize(word) for word in tweet.lower().split() if word not in stoplist and len(word) > 1]
                       for tweet in cleaned_data]
    
    cleaned_data = [" ".join(word) for word in cleaned_data]
    
    return cleaned_data

In [59]:
tesla_Scrapper = Scraper("VISA", 2)
#apple_Scrapper = Scraper("APPLE", 2)

In [1]:
#print(tesla_Scrapper.get())

In [61]:

#print(tesla_Scrapper.get(clean_data=False))

['#Summary\n#HomesForUkraine/#UkraineSponsorshipScheme\n#HFU #USS 🏠🇺🇦🇬🇧\nTo #May3 \n[with my "#visa #fairness/#share on the yellow background]\n\nsources:\nhttps://t.co/AQUoxtiGyD\nhttps://t.co/qnk3OECjLs https://t.co/vDmR0zMkUu', 'ETH Transactions Spiked 4.5x Surpassing Visa Payments in 2021\n\n#Digital_Payment #ETH  #Ethereum  #visa  #TheNewsCrypto #CryptoNews \n\nhttps://t.co/jRlFUfnDEh', '𝐍𝐞𝐰 𝐄𝐁-𝟓 𝐫𝐮𝐥𝐞𝐬 𝐫𝐞𝐪𝐮𝐢𝐫𝐞 𝐞𝐱𝐢𝐬𝐭𝐢𝐧𝐠 𝐑𝐞𝐠𝐢𝐨𝐧𝐚𝐥 𝐂𝐞𝐧𝐭𝐞𝐫𝐬 𝐭𝐨 𝐬𝐞𝐞𝐤 𝐑𝐞𝐜𝐞𝐫𝐭𝐢𝐟𝐢𝐜𝐚𝐭𝐢𝐨𝐧\n\n#UnitedStates #BusinessOwner #talent #USA #visa #immigration #travel #India #GreenCard #COVID19 #business #coronavirus #pandemic #legal #spring https://t.co/OKR08ikrhk', '#Visa Fear and Greed Index is currently 35 - FEAR\n\n24h technical analysis index: 29 - STRONG SELL\nRelative Strength Index #RSI: 23\nCurrent #V price is 205.248936579803 usd\n\nLive analysis for $V and other top 100 #stocks on the #stockmarket https://t.co/60MvwI9WeO', 'art for the eyes! https://t.co/K7o9p99xBY #monumentvalley #arizona #landscapelovers 

In [43]:
apple_Scrapper.get()

['currently safe enough used sole authentication method secure device irisrecognition iphone apple faceid watchguardtechnologies codebreaker thenameoftherose facialrecognition isoiec jtc sc',
 'apple settled class action lawsuit accused throttling iphone io update',
 'standard bedroom flat peace drive estate tank junction via pastortundebakare israel endsars apple cotonou umahi rap benin cancer portharcourt hellno ipob pfizerdocuments niger igbo rihanna houseofthedragon',
 'apple mac factory reset recovery process requires wired keyboard wired mouse please add support bluetooth keyboard mouse recovery software user click continue button apple mac',
 'automated hourly aapl sentiment report product series review iphones mac macbook ipad applewatch macos io general apple product company news social medium utc',
 'apple google microsoft mix effort towards passwordless logins alphabet apple google macbooks microsoft mobile technews technology window',
 'dino maker make dinos level share ind